In [1]:
#pip install gym

In [2]:
#pip install gym[atari]

In [3]:
#pip install gym[accept-rom-license]

In [4]:
#pip uninstall tensorflow

In [5]:
#!pip install keras-rl2

from ale_py import ALEInterface

ale = ALEInterface()
ALEInterface.isSupportedROM


!ale-import-roms roms/

In [6]:
#!pip install gym keras-rl2 gym[atari]

In [27]:
import gym
import random
env = gym.make('SpaceInvaders-v0')
height, width,channels =env.observation_space.shape 
actions = env.action_space.n

In [28]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [29]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [30]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [31]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
del model

In [33]:
model = build_model(height, width, channels, actions)

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 6)                

In [35]:
# original : 

In [36]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [37]:
from tensorflow import keras 
# initial lr 0.7
optimizer = keras.optimizers.Adam(learning_rate=0.7, clipnorm=1.0)

In [38]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.01, value_test=.2, nb_steps=100)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=100 , gamma=.8
                  )
    return dqn

In [39]:
# initial epsilon :1.0
dqn = build_agent(model, actions)
#dqn.compile(Adam(epsilon=0.7))
dqn.compile(optimizer)

In [40]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2,nb_max_episode_steps=99)

Training for 5000 steps ...
   99/5000: episode: 1, duration: 2.654s, episode steps:  99, steps per second:  37, episode reward: 10.000, mean reward:  0.101 [ 0.000, 10.000], mean action: 2.071 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  198/5000: episode: 2, duration: 99.151s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 3.949 [2.000, 4.000],  loss: inf, mean_q: 5702238046675566592.000000, mean_eps: 0.010000
  297/5000: episode: 3, duration: 102.739s, episode steps:  99, steps per second:   1, episode reward: 15.000, mean reward:  0.152 [ 0.000,  5.000], mean action: 3.960 [0.000, 4.000],  loss: inf, mean_q: 7507417715632123904.000000, mean_eps: 0.010000
  396/5000: episode: 4, duration: 99.459s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 4.000 [4.000, 4.000],  loss: inf, mean_q: 7471613352259362816.000000, mean_eps: 0.010000
  4

 3366/5000: episode: 34, duration: 100.943s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 3.960 [0.000, 4.000],  loss: inf, mean_q: 7459593418954357760.000000, mean_eps: 0.010000
 3465/5000: episode: 35, duration: 104.212s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 3.990 [1.000, 5.000],  loss: inf, mean_q: 7448408209332951040.000000, mean_eps: 0.010000
 3564/5000: episode: 36, duration: 100.610s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 4.000 [4.000, 4.000],  loss: inf, mean_q: 7443391609787976704.000000, mean_eps: 0.010000
 3663/5000: episode: 37, duration: 101.785s, episode steps:  99, steps per second:   1, episode reward: 10.000, mean reward:  0.101 [ 0.000,  5.000], mean action: 3.939 [1.000, 4.000],  loss: inf, mean_q: 7440654458886964224.000000, mean_eps: 0

In [41]:

scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 270.000, steps: 720
Episode 2: reward: 270.000, steps: 705
Episode 3: reward: 270.000, steps: 715
Episode 4: reward: 270.000, steps: 722
Episode 5: reward: 270.000, steps: 735
Episode 6: reward: 270.000, steps: 719
Episode 7: reward: 270.000, steps: 713
Episode 8: reward: 270.000, steps: 711
Episode 9: reward: 270.000, steps: 714
Episode 10: reward: 270.000, steps: 717
Episode 11: reward: 270.000, steps: 729
Episode 12: reward: 270.000, steps: 718
Episode 13: reward: 270.000, steps: 719
Episode 14: reward: 270.000, steps: 727
Episode 15: reward: 270.000, steps: 714
Episode 16: reward: 270.000, steps: 723
Episode 17: reward: 270.000, steps: 731
Episode 18: reward: 270.000, steps: 703
Episode 19: reward: 270.000, steps: 716
Episode 20: reward: 270.000, steps: 720
Episode 21: reward: 270.000, steps: 727
Episode 22: reward: 270.000, steps: 713
Episode 23: reward: 270.000, steps: 709
Episode 24: reward: 270.000, steps: 727
Episode 25: reward: 

In [42]:
dqn.save_weights('d:/TensorflowModule/SavedWeights/base_line_dif_alpha_gamma/dqn_weights.h5f')

NameError: name 'tensorflow' is not defined